# Lab 2: Swarm Intelligence

Implement swarm algorithms and observe emergent behavior.

## Learning Objectives
- Implement the Boids flocking algorithm
- Understand stigmergy through ant colony simulation
- Visualize emergent patterns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Part 1: Boids Flocking Algorithm

In [ ]:
class Boid:
    """A single boid in the flock"""
    
    def __init__(self, x: float, y: float, max_speed: float = 4.0):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.randn(2) * 2
        self.max_speed = max_speed
        self.max_force = 0.3
    
    def limit_magnitude(self, vector: np.ndarray, max_val: float) -> np.ndarray:
        mag = np.linalg.norm(vector)
        if mag > max_val:
            return vector / mag * max_val
        return vector
    
    def separation(self, neighbors: list, min_distance: float = 25) -> np.ndarray:
        """Avoid crowding neighbors"""
        steer = np.zeros(2)
        count = 0
        
        for other in neighbors:
            dist = np.linalg.norm(self.position - other.position)
            if 0 < dist < min_distance:
                diff = self.position - other.position
                diff = diff / dist  # Weight by distance
                steer += diff
                count += 1
        
        if count > 0:
            steer /= count
            steer = self.limit_magnitude(steer, self.max_force)
        
        return steer
    
    def alignment(self, neighbors: list) -> np.ndarray:
        """Match velocity with neighbors"""
        avg_velocity = np.zeros(2)
        count = len(neighbors)
        
        if count > 0:
            for other in neighbors:
                avg_velocity += other.velocity
            avg_velocity /= count
            avg_velocity = self.limit_magnitude(avg_velocity, self.max_speed)
            steer = avg_velocity - self.velocity
            return self.limit_magnitude(steer, self.max_force)
        
        return avg_velocity
    
    def cohesion(self, neighbors: list) -> np.ndarray:
        """Move toward center of neighbors"""
        center = np.zeros(2)
        count = len(neighbors)
        
        if count > 0:
            for other in neighbors:
                center += other.position
            center /= count
            desired = center - self.position
            desired = self.limit_magnitude(desired, self.max_speed)
            steer = desired - self.velocity
            return self.limit_magnitude(steer, self.max_force)
        
        return center
    
    def get_neighbors(self, all_boids: list, radius: float = 50) -> list:
        """Find boids within perception radius"""
        neighbors = []
        for other in all_boids:
            if other is not self:
                dist = np.linalg.norm(self.position - other.position)
                if dist < radius:
                    neighbors.append(other)
        return neighbors
    
    def update(self, all_boids: list, weights: dict, bounds: tuple):
        """Update position based on flocking rules"""
        neighbors = self.get_neighbors(all_boids)
        
        sep = self.separation(neighbors) * weights.get('separation', 1.5)
        ali = self.alignment(neighbors) * weights.get('alignment', 1.0)
        coh = self.cohesion(neighbors) * weights.get('cohesion', 1.0)
        
        self.velocity += sep + ali + coh
        self.velocity = self.limit_magnitude(self.velocity, self.max_speed)
        self.position += self.velocity
        
        # Wrap around edges
        self.position[0] = self.position[0] % bounds[0]
        self.position[1] = self.position[1] % bounds[1]

In [ ]:
class FlockSimulation:
    """Boids flock simulation"""
    
    def __init__(self, n_boids: int = 50, width: int = 100, height: int = 100):
        self.width = width
        self.height = height
        self.boids = [
            Boid(np.random.rand() * width, np.random.rand() * height)
            for _ in range(n_boids)
        ]
        self.weights = {
            'separation': 1.5,
            'alignment': 1.0,
            'cohesion': 1.0
        }
    
    def step(self):
        for boid in self.boids:
            boid.update(self.boids, self.weights, (self.width, self.height))
    
    def get_positions(self):
        return np.array([b.position for b in self.boids])
    
    def get_velocities(self):
        return np.array([b.velocity for b in self.boids])

# Run and visualize
sim = FlockSimulation(n_boids=40)

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)

scatter = ax.scatter([], [], c='blue', s=30)
quiver = ax.quiver([], [], [], [], scale=50)

def init():
    return scatter, quiver

def update(frame):
    sim.step()
    pos = sim.get_positions()
    vel = sim.get_velocities()
    
    scatter.set_offsets(pos)
    quiver.set_offsets(pos)
    quiver.set_UVC(vel[:, 0], vel[:, 1])
    
    return scatter, quiver

anim = FuncAnimation(fig, update, init_func=init, frames=100, interval=50, blit=True)
plt.title('Boids Flocking Simulation')
HTML(anim.to_jshtml())

## Exercise 1: Experiment with Weights

Try different weight combinations and observe the behavior:
- High separation, low cohesion: scattered flock
- Low separation, high cohesion: tight clusters
- High alignment: parallel movement

In [ ]:
# YOUR CODE HERE
# Create simulations with different weight configurations
# Compare the resulting patterns

## Part 2: Ant Colony Optimization

In [ ]:
class AntColony:
    """Ant Colony Optimization for TSP"""
    
    def __init__(self, distances: np.ndarray, n_ants: int = 10,
                 alpha: float = 1.0, beta: float = 2.0, evaporation: float = 0.5):
        self.distances = distances
        self.n_cities = len(distances)
        self.n_ants = n_ants
        self.alpha = alpha  # Pheromone importance
        self.beta = beta    # Distance importance
        self.evaporation = evaporation
        
        # Initialize pheromones
        self.pheromones = np.ones((self.n_cities, self.n_cities))
    
    def _transition_probability(self, current: int, unvisited: list) -> np.ndarray:
        """Calculate probability of moving to each unvisited city"""
        probs = np.zeros(len(unvisited))
        
        for i, city in enumerate(unvisited):
            tau = self.pheromones[current, city] ** self.alpha
            eta = (1.0 / self.distances[current, city]) ** self.beta
            probs[i] = tau * eta
        
        return probs / probs.sum()
    
    def _construct_path(self) -> list:
        """Ant constructs a path visiting all cities"""
        start = np.random.randint(self.n_cities)
        path = [start]
        unvisited = list(range(self.n_cities))
        unvisited.remove(start)
        
        while unvisited:
            probs = self._transition_probability(path[-1], unvisited)
            next_city = np.random.choice(unvisited, p=probs)
            path.append(next_city)
            unvisited.remove(next_city)
        
        return path
    
    def _path_distance(self, path: list) -> float:
        """Calculate total distance of path"""
        total = 0
        for i in range(len(path)):
            total += self.distances[path[i], path[(i+1) % len(path)]]
        return total
    
    def run(self, iterations: int = 100) -> tuple:
        """Run the optimization"""
        best_path = None
        best_distance = float('inf')
        history = []
        
        for iteration in range(iterations):
            # Each ant constructs a path
            paths = [self._construct_path() for _ in range(self.n_ants)]
            distances = [self._path_distance(p) for p in paths]
            
            # Update best
            min_idx = np.argmin(distances)
            if distances[min_idx] < best_distance:
                best_distance = distances[min_idx]
                best_path = paths[min_idx]
            
            history.append(best_distance)
            
            # Evaporate pheromones
            self.pheromones *= (1 - self.evaporation)
            
            # Deposit new pheromones
            for path, dist in zip(paths, distances):
                deposit = 1.0 / dist
                for i in range(len(path)):
                    j = (i + 1) % len(path)
                    self.pheromones[path[i], path[j]] += deposit
        
        return best_path, best_distance, history

# Create a random TSP problem
n_cities = 15
np.random.seed(42)
cities = np.random.rand(n_cities, 2) * 100

# Calculate distance matrix
distances = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(n_cities):
        distances[i, j] = np.linalg.norm(cities[i] - cities[j])

# Run ACO
aco = AntColony(distances, n_ants=20, alpha=1, beta=3, evaporation=0.3)
best_path, best_dist, history = aco.run(iterations=50)

print(f"Best distance: {best_dist:.2f}")
print(f"Best path: {best_path}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot path
ax = axes[0]
ax.scatter(cities[:, 0], cities[:, 1], c='red', s=100, zorder=5)
for i, city in enumerate(cities):
    ax.annotate(str(i), city, fontsize=8)

for i in range(len(best_path)):
    j = (i + 1) % len(best_path)
    ax.plot([cities[best_path[i], 0], cities[best_path[j], 0]],
            [cities[best_path[i], 1], cities[best_path[j], 1]], 'b-', linewidth=2)

ax.set_title('Best Path Found by ACO')

# Plot convergence
ax = axes[1]
ax.plot(history)
ax.set_xlabel('Iteration')
ax.set_ylabel('Best Distance')
ax.set_title('Convergence')
ax.grid(True)

plt.tight_layout()
plt.show()

## Exercise 2: Parameter Tuning

Experiment with ACO parameters:
- alpha (pheromone importance)
- beta (distance importance)
- evaporation rate

Find the best combination for the given problem.

In [ ]:
# YOUR CODE HERE
# Test different parameter combinations
# Compare convergence speed and solution quality